In [ ]:
%matplotlib widget
from lib.preamble import *

In [ ]:
paths = glob("../data/2023-10-31/d*.asc")
paths += glob("../data/2023-10-30/d*.asc")

# import mirrors
paths += glob("../data/2023-11-02/d002_mirror_2x5s.asc")
paths += glob("../data/2023-11-02/d003_mirror_2x5s-defocused.asc")

In [ ]:
df = pd.DataFrame(map(import_spectrum, paths)).set_index("material").sort_index()
df

# Display raw spectra


In [ ]:
sel = df
sel = sel[sel.method=="luminescence"]
# sel = sel[sel.method=="reflectance"]
sel = sel[sel.index=="NiPS3"]
fig, ax = plot.rows(sel, "wavelength", "intensity", "name")
plot.energy_ticks(ax[0])
plt.tight_layout()
plt.subplots_adjust(hspace=0)
plt.show()

# Calculate layer thickness

In [ ]:
thicknesses = np.geomspace(10, 1000, 1000)  # um
df["thickness"] = df.apply(lambda x: calc_thickness(x["wavelength"], x["intensity"], thicknesses*1e3), axis=1)

In [ ]:
fig, ax = plot.stack(len(df), figsize=(10, 5))
for a, (m, row) in zip(ax, df.iterrows()):
    a.plot(thicknesses, row.thickness)
    a.set_ylabel(m)
    a.set_xscale("log")
plot.footnote(ax,df.name)
fig.tight_layout()
plt.subplots_adjust(hspace=0)
plt.show()

# Reflection

In [ ]:
# Reflection
sel = df.loc[df.index == "NiPS3"]
sel = sel[sel.method=="reflectance"]
fig, ax = plot.rows(sel, "wavelength", "intensity", "name")
plot.energy_ticks(ax[0])
for a in ax:
    a.set_yticks([])
plt.tight_layout()
plt.subplots_adjust(hspace=0)
plt.show()

# Normalize with Mirror

In [ ]:
from scipy.interpolate import interp1d

df["intensity_interp"] = df.apply(
    lambda x: interp1d(x.wavelength, x.intensity, fill_value="extrapolate"),
 axis=1)

In [ ]:
mirror = df.loc["Cu"]

df["normalized"] = df.apply(
    lambda x: x.intensity / mirror.intensity_interp(x.wavelength),
 axis=1)

sel = df.loc[df.index != mirror.name]
fig, ax = plot.rows(sel, "wavelength", "normalized", "name", figsize=(10, 5))
plot.energy_ticks(ax[0])
ax[0].set_title(f"Normalized with {mirror.name} Spectrum")
for a in ax:
    a.set_yticks([])
plt.tight_layout()
plt.subplots_adjust(hspace=0)
plt.show()

In [ ]:
mirrors = df.loc[["Cu", "mirror"]]
sel = df.loc[df.index == "NiPS3"]

fig, ax = plot.stack(len(sel))

for a, (m, row) in zip(ax, sel.iterrows()):
    for mir_i, mir in mirrors.iterrows():
        # a.plot(row.wavelength, row.intensity / mir.intensity_interp(row.wavelength), label=mir.name)
        a.plot(row.wavelength, row.intensity / mir.intensity_interp(row.wavelength) * np.max(mir.intensity_interp(row.wavelength)), label=mir.name)
    a.set_ylabel(m)
    a.legend()

plot.energy_ticks(ax[0])
ax[0].set_title(f"Normalized")
for a in ax:
    a.set_yticks([])
plt.tight_layout()
plt.subplots_adjust(hspace=0)
plt.show()

# Try Xarray

In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv
hv.extension("bokeh", width=100)

In [ ]:
paths = glob("../data/*/d*.asc")
paths = sorted(paths)
data = {p:import_spectrum(p) for p in paths}

dataArrays = {
	v["name"]: xr.DataArray(
		data=v["intensity"],
		dims=["wavelength"],
		coords={
			"wavelength": v["wavelength"],
			"material": v["material"],
			"method": v["method"],
			"experiment": v["experiment"],
		},
		attrs=v,
		name=v["name"]
	)
	for _,v in data.items()
}

# ds = xr.Dataset(dataArrays)
# combine all
ds = xr.concat(dataArrays.values(), dim="name")

In [ ]:
ds

In [ ]:
list(dataArrays.values())[0].hvplot("wavelength")

In [ ]:
ds.hvplot.line("wavelength", group=["material", "method"])